# Server

In [ ]:
# import library untuk SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCServer

# import library untuk SimpleXMLRPCRequestHandler
from xmlrpc.server import SimpleXMLRPCRequestHandler

# import library untuk threading locking
import threading

# import library untuk read CSV
import pandas as pd

# import library untuk random
import numpy as np

# import library untuk date
import datetime

In [ ]:
# Definisikan IP address dan port
IP = 'localhost'
PORT = 8080

# Batasi path (folder) yang bisa diakses
class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

## Data struktur untuk movie dan theater

In [ ]:
df = pd.read_csv("movies.csv")

In [ ]:
movies = df.head()
movies

In [ ]:
theaters = [
    {
        'theaterId' : 1,
        'name' : 'Theater 1',
        'price' : 30000
    },
    {
        'theaterId' : 2,
        'name' : 'Theater 2',
        'price' : 60000
    }
]

theaters = pd.DataFrame(data=theaters)
theaters

## Data struktur untuk seat dan schedule

In [ ]:
schedules = [
    {
        'scheduleId' : 1,
        'movieId' : 1,
        'theaterId' : 1,
        'seat' : np.random.randint(2, size=10).tolist(),
        'date' : datetime.date(2020, 3, 12).strftime("%d %B %Y"),
        'time' : datetime.time(10, 0).strftime("%H:%M")
    },
    {
        'scheduleId' : 2,
        'movieId' : 2,
        'theaterId' : 2,
        'seat' : np.random.randint(2, size=10).tolist(),
        'date' : datetime.date(2020, 3, 12).strftime("%d %B %Y"),
        'time' : datetime.time(10, 0).strftime("%H:%M")
    }
]

schedules = pd.DataFrame(data=schedules)
schedules

## Buat server

In [ ]:
with SimpleXMLRPCServer((IP, PORT),
                        requestHandler=RequestHandler) as server:
    server.register_introspection_functions()
    
    # assign threading.Lock() karena ada fungsi yang critical section yang akan menyebabkan race condition
    lock = threading.Lock()
    
    # Fungsi untuk melihat jadwal film saat ini
    def movie_schedule():
        # lock terlebih dahulu karena critical section dibawah
        lock.acquire()
        
        data_schedule = []
        for index, row in schedules.iterrows():
            dat = {
                'id schedule' : row['scheduleId'],
                'title' : movies[movies['movieId'] == row['movieId']]['title'][index],
                'theater' : theaters[theaters['theaterId'] == row['theaterId']]['name'][index],
                'date' : row['date'],
                'time' : row['time'],
                'seat' : row['seat'],
                'tickets remaining' : str(10 - np.sum(row['seat'])),
                'price': 'Rp. ' + str(theaters[theaters['theaterId'] == row['theaterId']]['price'][index])
            }
            data_schedule.append(dat)
        
        # lock dibuka karena critical section sudah selesai
        lock.release()
        
        return data_schedule
    
    # Registrasi fungsi movie_schedule() ke server
    server.register_function(movie_schedule)
    
    # Fungsi untuk membeli tiket bioskop
    def buy_ticket(scheduleId=-1, seat_num=-1):
               
        message = ''
        
        if (seat_num < 1 or seat_num > 10):
            return 'seat_num range between 1 and 10'
        
        seat_num -= 1
        
        data = schedules[schedules['scheduleId'] == scheduleId]
        if (len(data) == 0):
            return 'Schedule not found'
        
        # lock terlebih dahulu karena critical section dibawah
        lock.acquire()
        
        seat = data['seat'].tolist()[0]
        if (seat[seat_num] == 0):
            seat[seat_num] = 1
            schedules.at[data.index.values[0], 'seat'] = seat
            
            message = 'Thank you for buying ticket'
        else:
            message = 'Seat has been taken'
        
        # lock dibuka karena critical section sudah selesai
        lock.release()
        
        return message
        
    # Registrasi fungsi buy_ticket(scheduleId=-1, seat_num=-1) ke server
    server.register_function(buy_ticket)
    
    # Fungsi admin untuk melihat jadwal film saat ini 
    def admin_movie_schedule(user, password):
        if (user == 'admin' and password == 'admin'):
            return schedules.values.tolist()
        else:
            return 'username or password is wrong'
        
    # Registrasi fungsi admin_movie_schedule(user, password) ke server
    server.register_function(admin_movie_schedule)
    
    # Fungsi admin untuk melihat daftar film
    def admin_movies(user, password):
        if (user == 'admin' and password == 'admin'):
            return movies.values.tolist()
        else:
            return 'username or password is wrong'
        
    # Registrasi fungsi admin_movies(user, password) ke server
    server.register_function(admin_movies)
    
    # Fungsi admin untuk melihat daftar theather saat ini 
    def admin_theaters(user, password):
        if (user == 'admin' and password == 'admin'):
            return theaters.values.tolist()
        else:
            return 'username or password is wrong'
        
    # Registrasi fungsi admin_theaters(user, password) ke server
    server.register_function(admin_theaters)
    
    # Fungsi admin untuk menambah jadwal film
    def admin_add_schedule(request, user, password):
        if (user != 'admin' and password != 'admin'):
            return 'username or password is wrong'
        
        # lock terlebih dahulu karena critical section dibawah
        lock.acquire()
        
        global schedules
        
        date = request['date'].split('-')
        time = request['time'].split(':')
        
        data = {
            'scheduleId' : int(schedules.scheduleId.values[-1]) + 1,
            'movieId' : request['movieId'],
            'theaterId' : request['theaterId'],
            'seat' : [0 for i in range(10)],
            'date' : datetime.date(int(date[2]), int(date[1]), int(date[0])).strftime("%d %B %Y"),
            'time' : datetime.time(int(time[0]), int(time[1])).strftime("%H:%M")
        }

        schedules = schedules.append(data, ignore_index=True)
        
        # lock dibuka karena critical section sudah selesai
        lock.release()
        
        return 'Add successful'
        
    # Registrasi fungsi admin_add_schedule(request, user, password) ke server
    server.register_function(admin_add_schedule)
    
    # Fungsi admin untuk mengedit jadwal film
    def admin_edit_schedule(request, user, password):
        if (user != 'admin' and password != 'admin'):
            return 'username or password is wrong'
        
        data = schedules[schedules['scheduleId'] == request['scheduleId']]
        if (len(data) == 0):
            return 'Schedule not found'
    
        # lock terlebih dahulu karena critical section dibawah
        lock.acquire()
        
        if request['movieId'] != -1:
            schedules.at[data.index.values[0], 'movieId'] = request['movieId']
            
        if request['theaterId'] != -1:
            schedules.at[data.index.values[0], 'theaterId'] = request['theaterId']
            
        if request['date'] != -1:
            date = request['date'].split('-')
            schedules.at[data.index.values[0], 'date'] = datetime.date(int(date[2]), int(date[1]), int(date[0])).strftime("%d %B %Y")
            
        if request['time'] != -1:
            time = request['time'].split(':')
            schedules.at[data.index.values[0], 'time'] = datetime.time(int(time[0]), int(time[1])).strftime("%H:%M")
        
        # lock dibuka karena critical section sudah selesai
        lock.release()
        
        return 'Edit schedule is successful'
    
    # Registrasi fungsi admin_edit_schedule(request, user, password) ke server
    server.register_function(admin_edit_schedule)
    
    # Fungsi admin untuk mendelete jadwal film
    def admin_del_schedule(scheduleId, user, password):
        if (user != 'admin' and password != 'admin'):
            return 'username or password is wrong'

        # lock terlebih dahulu karena critical section dibawah
        lock.acquire()
        
        global schedules
        schedules = schedules[schedules.scheduleId != scheduleId]
        
        # lock dibuka karena critical section sudah selesai
        lock.release()
        
        return 'Delete successful'
        
    # Registrasi fungsi admin_del_schedule(scheduleID, user, password) ke server
    server.register_function(admin_del_schedule)
    
    # Fungsi admin untuk mengedit harga tiket
    def admin_change_price(theaterId, new_price, user, password):
        if (user != 'admin' and password != 'admin'):
            return 'username or password is wrong'
        
        data = theaters[theaters['theaterId'] == theaterId]
        if (len(data) == 0):
            return 'Theaters not found'
    
        # lock terlebih dahulu karena critical section dibawah
        lock.acquire()
        
        theaters.at[data.index.values[0], 'price'] = new_price
        
        # lock dibuka karena critical section sudah selesai
        lock.release()
        
        return 'Change price is successful'
    
    # Registrasi fungsi admin_change_price(theaterId, new_price, user, password) ke server
    server.register_function(admin_change_price)
        
    print ("Server no voting ga starto desuyooo....")
    # Server START
    server.serve_forever()